In [1]:
from pymongo import MongoClient
from concurrent.futures import ThreadPoolExecutor
from threading import Lock
import time
import math
import matplotlib.pyplot as plt
import os

Конфігурація та підключення до бази даних

In [2]:
lock = Lock()

MONGODB_URI = os.environ.get("MONGODB_URI", "mongodb://localhost:27017")
client = pymongo.MongoClient(MONGODB_URI, username="root", password="root")
db = client.cafes
cafes = db.cafes

Тестові операції для роботи з СУБД

In [3]:

def transfer_funds(from_account, to_account, amount):
    cafe = cafes.find_one({'name': from_account})
    from pprint import pprint
    pprint(cafe)
    
    lock.acquire()
    cafes.update_one({'name': from_account}, {'$inc': {'rating': -amount}})
    cafes.update_one({'name': to_account}, {'$inc': {'rating': amount}})
    lock.release()

def credit_funds(to_account_name, amount):
    lock.acquire()
    cafes.update_one({'name': to_account_name}, {'$inc': {'rating': amount}})
    lock.release()

Бенчмарк

In [4]:
def run_transactions(num_threads, num_transactions):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        start_time = time.time()
        
        for _ in range(num_transactions):
            futures.append(executor.submit(lambda: transfer_funds(1, 2, 10)))
            futures.append(executor.submit(lambda: transfer_funds(2, 3, 10)))
            futures.append(executor.submit(lambda: transfer_funds(3, 4, 10)))

        for _ in range(math.floor(num_transactions / 2)):
            futures.append(executor.submit(lambda: credit_funds('Max', 10)))
            futures.append(executor.submit(lambda: credit_funds('John', 10)))
        
        for future in futures:
            future.result()

        executor.shutdown()
        
        end_time = time.time()
        return end_time - start_time

Main

In [ ]:
num_threads = 50
num_transactions = 100
iterations = 50
times = []
for i in range(iterations):
    time_taken = run_transactions(num_threads, num_transactions)
    times.append(time_taken)
    print(f'Completed {num_transactions + num_transactions / 2} transactions in {i+1} iteration in {time_taken:.4f} seconds')

Побудова графіків

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20.5, 9.5)
ax.plot(range(1, iterations+1), times, label='htm')
ax.set_xlabel('Iteration')
ax.set_ylabel('Time taken (seconds)')

ax.set_title('hardware transactional memory')
ax.legend()
plt.show()